## Reference to https://github.com/bluesy7585/tensorflow_LPRnet

## Monitor GPU

In [ ]:
!nvidia-smi

## Start training

In [ ]:
!~/.conda/envs/conda-py36env/bin/python main.py -m train

## Testing with single image

In [ ]:
!~/.conda/envs/conda-py36env/bin/python main.py -m test \
    -c ~/CarCarder/tensorflow_LPRnet/checkpoint/LPRnet_steps100000_loss_0.000.ckpt \
    --img ~/CarCarder/tensorflow_LPRnet/test/MLR-6733_12312.jpg

## Tensorboard

In [ ]:
# Run it one time

%load_ext tensorboard

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

g = tf.Graph()

with g.as_default() as g:
    tf.train.import_meta_graph('/home/mlst01/CarCarder/tensorflow_LPRnet/finalCKP/LPRnet_steps100000_loss_0.000.ckpt.meta')

with tf.Session(graph=g) as sess:
    file_writer = tf.summary.FileWriter(logdir='/home/mlst01/CarCarder/tensorflow_LPRnet/tensorboard', graph=g)

In [ ]:
#%conda activate conda-py36env
!~/.conda/envs/conda-py36env/bin/tensorboard --logdir tensorboard

## Frozen to freeze_grahp.pb

In [ ]:
!~/.conda/envs/conda-py36env/bin/python freeze.py \
    --model_dir ~/CarCarder/tensorflow_LPRnet/finalCKP \
    --output_node_names conv_out

## Convert to tflite

In [ ]:
import os
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

trained_checkpoint_prefix = '/home/mlst01/CarCarder/tensorflow_LPRnet/finalCKP/LPRnet_steps100000_loss_0.000.ckpt'
export_dir = os.path.join('/home/mlst01/CarCarder/tensorflow_LPRnet/finalCKP', '0')

graph = tf.Graph()
with tf.compat.v1.Session(graph=graph) as sess:
    # Restore from checkpoint
    loader = tf.compat.v1.train.import_meta_graph(trained_checkpoint_prefix + '.meta')
    loader.restore(sess, trained_checkpoint_prefix)

    # Export checkpoint to SavedModel
    builder = tf.compat.v1.saved_model.builder.SavedModelBuilder(export_dir)
    builder.add_meta_graph_and_variables(sess,
                                         [tf.saved_model.TRAINING, tf.saved_model.SERVING],
                                         strip_default_attrs=True)
    builder.save()

In [ ]:
import tensorflow as tf

# Convert the model.
converter = tf.compat.v1.lite.TFLiteConverter.from_frozen_graph(
    graph_def_file='/home/mlst01/CarCarder/tensorflow_LPRnet/finalCKP/frozen_graph.pb',
                    # both `.pb` and `.pbtxt` files are accepted.
    input_arrays=['inputs'],
    input_shapes={'inputs' : [1, 24, 94, 3]},
    output_arrays=['decoded']
)
converter.allow_custom_ops = True
converter.experimental_new_converter = True
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS,
                    tf.lite.OpsSet.SELECT_TF_OPS]
tflite_model = converter.convert()


# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
!saved_model_cli show --dir /home/mlst01/CarCarder/tensorflow_LPRnet/finalCKP/0 --tag_set serve